In [7]:
import json
import csv
count=0

with open('dialogues_001.json') as data_file:    
    data = json.load(data_file)
    # with open ('output.txt', 'w') as file:
    with open ('sample.tsv','wt') as file:
        tsv_writer = csv.writer(file, delimiter='\t')
        tsv_writer.writerow(['trg','src'])
        for conversation in data:
            services=conversation['services']
            domain="<"+"><".join(services)+">"
            domain=f"<domain> {domain} <domain>"
            dialogues=conversation['turns']
            history=""
            dialogue_len=len(dialogues)
            for iter,turn in enumerate(dialogues):
                speaker=turn['speaker']
                dialogue=turn['utterance']
                if speaker=="USER":
                    cur_utterance=f"<cur_utterance> {dialogue} <cur_utterance>"
                    cur_history=f"<history> {history} <history>"
                    if iter+1<dialogue_len :
                        next_speaker=dialogues[iter+1]['speaker']
                        if next_speaker=="SYSTEM":
                            src=domain+"\t"+cur_history+"\t"+cur_utterance
                            trg=dialogues[iter+1]['utterance']
                            tsv_writer.writerow([src,trg])
                    history+=f"<u> {dialogue} <u>"
                elif speaker=="SYSTEM":
                    history+=f"<s> {dialogue} <s>"
            
        
            


In [53]:
import torch.nn as nn
import torch
import numpy as np
def cosine_similarity(output, target):
    # output_tensor=torch.tensor(output,requires_grad=True)
    output_tensor=output
    target_tensor=torch.tensor(target)
    loss = torch.dot(output_tensor,target_tensor)
    loss = loss/(torch.norm(output_tensor)*torch.norm(target_tensor))
    return loss

model = nn.Linear(2, 2)
x = torch.randn(2)
target = torch.randn(2)
target = target.numpy()
# output = torch.randn(2)
# output= output.numpy()
output=model(x)
loss = cosine_similarity(output, target)
loss.backward()
print(model.weight.grad)



tensor([[-0.1243,  0.1850],
        [ 0.1433, -0.2133]])


In [14]:
import os
import json
import csv
def extract(actual_filename,tsv_writer):
    with open(actual_filename) as data_file:    
        data = json.load(data_file)
        for conversation in data:
            services=conversation['services']
            domain=",".join(services)
            domain=f"<domain> {domain} <domain>"
            dialogues=conversation['turns']
            history=""
            dialogue_len=len(dialogues)
            for iter,turn in enumerate(dialogues):
                speaker=turn['speaker']
                dialogue=turn['utterance']
                if speaker=="USER":
                    cur_utterance=f"<cur_utterance> {dialogue} <cur_utterance>"
                    cur_history=f"<history> {history} <history>"
                    if iter+1<dialogue_len :
                        next_speaker=dialogues[iter+1]['speaker']
                        if next_speaker=="SYSTEM":
                            src=domain+cur_history+cur_utterance #/t removed
                            trg=dialogues[iter+1]['utterance']
                            tsv_writer.writerow([src,trg])
                    history+=f"<u> {dialogue} <u>"
                elif speaker=="SYSTEM":
                    history+=f"<s> {dialogue} <s>"
                

In [15]:
#this code is ran at LE_S folder
folder_list=['dev','test','train']
# folder_list=['train']
parent_directory='/mnt/Data/abhisek_1921cs16/R/CRW/1.NLG_LE/LE_S/Datasets/T/MultiWOZ_2.2/'
for folder in folder_list:
    with open (folder+'.tsv', 'w') as file:
        tsv_writer = csv.writer(file, delimiter='\t')
        # tsv_writer.writerow(['src','trg'])
        directory=parent_directory+folder
        for filename in os.listdir(directory):
            actual_filename=directory+'/'+filename 
            extract(actual_filename,tsv_writer)

In [4]:
#extracting in text file
import os
import json
import csv
def extract(actual_filename,src_file,trg_file):
    with open(actual_filename) as data_file:    
        data = json.load(data_file)
        for conversation in data:
            services=conversation['services']
            domain=",".join(services)
            domain=f"<domain> {domain} <domain>"
            dialogues=conversation['turns']
            history=""
            dialogue_len=len(dialogues)
            for iter,turn in enumerate(dialogues):
                speaker=turn['speaker']
                dialogue=turn['utterance']
                if speaker=="USER":
                    cur_utterance=f"<cur_utterance> {dialogue} <cur_utterance>"
                    cur_history=f"<history> {history} <history>"
                    if iter+1<dialogue_len :
                        next_speaker=dialogues[iter+1]['speaker']
                        if next_speaker=="SYSTEM":
                            src=domain+cur_history+cur_utterance #/t removed
                            trg=dialogues[iter+1]['utterance']
                            src_file.write(src+'\n')
                            trg_file.write(trg+'\n')
                    history+=f"<u> {dialogue} <u>"
                elif speaker=="SYSTEM":
                    history+=f"<s> {dialogue} <s>"
                
#this code is ran at LE_S folder
folder_list=['dev','test','train']
# folder_list=['train']
parent_directory='/mnt/Data/abhisek_1921cs16/R/CRW/1.NLG_LE/LE_S/Datasets/T/MultiWOZ_2.2/'
for folder in folder_list:
    with open (folder+'_src.txt','w') as src_file, open (folder+'_trg.txt','w') as trg_file:
        directory=parent_directory+folder
        for filename in os.listdir(directory):
            actual_filename=directory+'/'+filename 
            extract(actual_filename,src_file,trg_file)

In [5]:
# 2.paste -d'\t' src_file trg_file label_file > train.tsv
import os 
files=['train','dev','test']
# paste -d'\t' train_src.txt train_trg.txt > train.tsv
for file in files:
    os.system(f"paste -d'\t' {file}_src.txt {file}_trg.txt > {file}.tsv")
os.system("python preprocess_parallel_corpus_labels.py")


Preprocessed file written: train.tsv.pre
No. of lines truncated: 0
Preprocessed file written: dev.tsv.pre
No. of lines truncated: 0
Preprocessed file written: test.tsv.pre
No. of lines truncated: 0


0

In [10]:
import pandas as pd
import csv
import numpy as np
counts=[]
# read data from file
# interviews_df = pd.read_csv('train.tsv', sep='\t')

c=0
# printing data)
with open('dev.tsv') as file:
    tsv_file = csv.reader(file, delimiter="\t")
    for line in tsv_file:
        counts.append(len(line[0].split()))

import numpy as np
np.percentile(counts,99)

In [3]:
def format(cur,pattern):
    return pattern+cur+pattern

def extract_pattern(dialogue,pattern):
    list=[]
    flag=0
    temp=""
    for index,i in enumerate(dialogue):
        if dialogue[index:index+len(pattern)]==pattern:
            if flag==0:
                flag=1
            else:
                list.append(temp[len(pattern):])
                temp=""
                flag=0
        if flag:
            temp+=i
    return list

def convert_to_last3(dialogue):
    user=extract_pattern(dialogue,"<u>")
    system=extract_pattern(dialogue,"<s>")
    domain=extract_pattern(dialogue,"<domain>")
    cur_utterance=extract_pattern(dialogue,"<cur_utterance>")
    total=len(user)
    user=user[-(min(3,total)):]
    system=system[-min(3,total):]

    new_dialogue=format(domain[0],"<domain>")
    for cur_user,cur_system in zip(user,system):
        new_dialogue+=format(cur_user,"<u>")
        new_dialogue+=format(cur_system,"<s>")
    new_dialogue+=" <history>"
    new_dialogue+=format(cur_utterance[0],"<cur_utterance>")
    return new_dialogue

files=["dev","train","test"]
for file_name in files:
    writer_file=file_name+"_src1.txt"
    filename=file_name+"_src.txt"
    with open(filename,"r") as reader,open(writer_file,"w") as writer:
        for line in reader:
            temp=convert_to_last3(line)
            writer.write(temp+"\n")

In [5]:
import re
files=['dev_src.txt','train_src.txt','test_src.txt']
count=0
cur=""
def convert(text):
    regex = r'<domain>(.*?)<domain>'
    match = re.search(regex, text)
    if match:
        return match.group(1)
for file in files:
    with open(file,'r') as reader_file:
        for line in reader_file:
            temp=convert(line)
            if temp!=cur:
                count+=1
                cur=temp
print(count)


9575
